In [34]:
import pandas as pd
import numpy as np
import ast

In [35]:
# Carregando os DataFrames
df_filmes = pd.read_csv('data/movies.csv', low_memory=False)
df_creditos = pd.read_csv('data/credits.csv')

### Pré processamento de dados

In [36]:
# Exibição dos DataFrames
print(df_filmes.info())
print(df_creditos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [37]:
# Concateção dos DataFrames
df_filmes = df_filmes.merge(df_creditos, on='title') # on='title' -> chave de junção
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

In [38]:
# Seleção de colunas relevantes
df_filmes = df_filmes[['movie_id','title', 'overview','genres', 'cast', 'keywords', 'crew']] 

In [39]:
# Verificação de valores nulos e remoção
print(df_filmes.isnull().sum())
df_filmes.dropna(inplace=True)

# Presemça de valores duplicados
print('Valores duplicados: ', df_filmes.duplicated().sum())


movie_id    0
title       0
overview    3
genres      0
cast        0
keywords    0
crew        0
dtype: int64
Valores duplicados:  0


In [40]:
# Tratamento para converter as colunas em listas
def convert_col_to_list(col):
    """Converte a coluna em uma lista"""
    if isinstance(col, list):
        return [i['name'] for i in col]
    else:
        return []

df_filmes['genres'] = df_filmes['genres'].apply(ast.literal_eval).apply(convert_col_to_list) # ast.literal_eval -> converte a string em lista
df_filmes['cast'] = df_filmes['cast'].apply(ast.literal_eval).apply(convert_col_to_list)
df_filmes['keywords'] = df_filmes['keywords'].apply(ast.literal_eval).apply(convert_col_to_list)

# Filtra o nome do diretor
def get_diretor(col):
    """Filtra o nome do diretor"""
    for i in col:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

df_filmes['director'] = df_filmes['crew'].apply(ast.literal_eval).apply(get_diretor)
df_filmes.drop('crew', axis=1, inplace=True)

In [41]:
# Título em maiusculo
df_filmes['title'] = df_filmes['title'].str.upper()

In [42]:
# Conversão de overview e director para lista
df_filmes['overview'] = df_filmes['overview'].apply(lambda x: x.split())
df_filmes['director'] = df_filmes['director'].apply(lambda x: [x])

In [43]:
# Remoção de espaços em branco
def remove_space(col):
    """Remove os espaços em branco"""
    if isinstance(col, list):
        return [str.lower(i.replace(" ", "")) for i in col]
    else:
        if isinstance(col, str):
            return str.lower(col.replace(" ", ""))
        else:
            return ''

# Aplicação da função
df_filmes['genres'] = df_filmes['genres'].apply(remove_space)
df_filmes['cast'] = df_filmes['cast'].apply(remove_space)
df_filmes['keywords'] = df_filmes['keywords'].apply(remove_space)
df_filmes['director'] = df_filmes['director'].apply(remove_space) if isinstance(df_filmes['director'], str) else df_filmes['director']


In [44]:
# Criação de uma coluna com os dados concatenados
df_filmes['metadata'] = df_filmes['overview'] + df_filmes['genres'] + df_filmes['keywords'] + df_filmes['cast'] + df_filmes['director']

In [45]:
# Cria um novo DataFrame com as colunas relevantes
df_filmes = df_filmes[['movie_id','title','metadata']]

In [46]:
# Restauração dos espaços em branco
df_filmes['metadata'] = df_filmes['metadata'].apply(lambda x: ' '.join(str(i) for i in x)) # str(i) -> converte o valor em string para poder usar o join

In [47]:
# Transformação dos dados para minúsculo
df_filmes['metadata'] = df_filmes['metadata'].str.lower()

### Análise de linguagem natural

In [48]:
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer -> cria uma matriz de contagem de token
from sklearn.metrics.pairwise import cosine_similarity # cosine_similarity -> calcula a similaridade entre os vetores

count_vec = CountVectorizer(max_features= 5000 ,stop_words='english') # stop_words='english' -> remove as palavras 'inúteis' em inglês
vetores = count_vec.fit_transform(df_filmes['metadata']).toarray() # fit_transform -> transforma os dados em uma matriz de contagem de token

In [49]:
# Uso de linguagem natural para encontrar a similaridade entre os filmes
from nltk.stem.porter import PorterStemmer # PorterStemmer -> reduz as palavras para o seu radical

port_strem = PorterStemmer()

def stemmer(col):
    """Reduz as palavras para o seu radical"""
    return [port_strem.stem(i) for i in col]

# Aplicação da função
df_filmes['metadata'] = df_filmes['metadata'].apply(lambda x: stemmer(x.split()))

In [50]:
similaridade = cosine_similarity(vetores) # Calcula a similaridade entre os vetores

# Ordenação dos filmes por similaridade
def recomendacao_filmes(filme):
    """Recomenda os filmes com base na similaridade"""
    filme = filme.upper()
    if filme not in df_filmes['title'].unique():
        return 'Filme não encontrado'
    else:
        index_filme = df_filmes.loc[df_filmes['title'] == filme].index[0]
        filmes_similares = sorted(list(enumerate(similaridade[index_filme])), key=lambda x: x[1], reverse=True)[1:11]
        for i in filmes_similares:
            print(df_filmes.iloc[i[0]]['title'])

### Sugestões de filmes

In [51]:
# Exibição dos filmes recomendados para o filme Bastardos Inglórios
recomendacao_filmes('Inglourious Basterds')

BLACK BOOK
SAINTS AND SOLDIERS
SCHINDLER'S LIST
U-571
WOMAN IN GOLD
FURY
RED TAILS
THE TRAIN
THE GREAT ESCAPE
GODS AND GENERALS
